In [1]:
import os
import torch
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.utils.data import DataLoader
from modules.lifter_2d_3d.dataset.drive_and_act_keypoint_dataset import DriveAndActKeypointDataset
from modules.lifter_2d_3d.model.linear_model.lit_linear_model import LitSimpleBaselineLinear
from modules.utils.visualization import (
    plot_samples
)
from IPython.display import display
from pathlib import Path

pl.seed_everything(1234)

# ------------
# dataset path
# ------------
dataset_root_path = Path('/root/data/processed/drive_and_act/')
keypoint_2d_path = dataset_root_path / 'keypoint_detection_results'
keypoint_3d_path = dataset_root_path / 'annotations'
# ------------
# model
# ------------
image_width = 1280
image_height = 1024
batch_size = 64
max_epoch = 200
val_check_period = 5
early_stopping_patience = 5

Global seed set to 1234


In [2]:
all_actors = ['vp1', 'vp2', 'vp3', 'vp4', 'vp5', 'vp6', 'vp7', 'vp8', 'vp9', 'vp10']
all_mpjpe = []
all_pjpe = []
all_activity_mpjpe = []


for actor in all_actors:
    saved_model_path = f'./saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model_single_actor/actor_{actor}'
    train_dataset = DriveAndActKeypointDataset(
        prediction_file=(keypoint_2d_path / 'keypoint_detection_train.json').as_posix(),
        annotation_file=(keypoint_3d_path / 'person_keypoints_train.json').as_posix(),
        image_width=image_width,
        image_height=image_height,
        actors=[actor],
        exclude_ankle=True,
        exclude_hip=True
    )
    val_dataset = DriveAndActKeypointDataset(
        prediction_file=(keypoint_2d_path / 'keypoint_detection_val.json').as_posix(),
        annotation_file=(keypoint_3d_path / 'person_keypoints_val.json').as_posix(),
        image_width=image_width,
        image_height=image_height,
        actors=['vp11', 'vp12'],
        exclude_ankle=True,
        exclude_hip=True
    )
    test_dataset = DriveAndActKeypointDataset(
        prediction_file=(keypoint_2d_path / 'keypoint_detection_test.json').as_posix(),
        annotation_file=(keypoint_3d_path / 'person_keypoints_test.json').as_posix(),
        image_width=image_width,
        image_height=image_height,
        actors=['vp13', 'vp14', 'vp15'],
        exclude_ankle=True,
        exclude_hip=True
    )

    lit_model = LitSimpleBaselineLinear(exclude_ankle=True, exclude_hip=True, all_activities=train_dataset.activities, is_silence=True)
    print(
        'train_dataset', len(train_dataset),
        'val_dataset', len(val_dataset),
        'test_dataset', len(test_dataset)
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True, shuffle=True, num_workers=24)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, drop_last=True, num_workers=24)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=24)

    model_checkpoint = ModelCheckpoint(monitor='val_loss',mode='min', save_top_k=1)
    early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=early_stopping_patience)

    if not os.path.exists(saved_model_path):
        os.makedirs(saved_model_path)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    trainer = pl.Trainer(
        # max_steps=10,
        max_epochs=max_epoch,
        callbacks=[model_checkpoint, early_stopping],
        accelerator=device,
        check_val_every_n_epoch=val_check_period,
        default_root_dir=saved_model_path,
        gradient_clip_val=1.0,
        enable_progress_bar = False
    )
    trainer.fit(lit_model, train_loader, val_loader)
    with open(f'{saved_model_path}/best_model_path.txt', 'w') as f:
        f.writelines(model_checkpoint.best_model_path)
    best_checkpoint_path = model_checkpoint.best_model_path
    trainer.test(ckpt_path=best_checkpoint_path, dataloaders=test_loader)
    all_mpjpe.append(lit_model.test_history[0]['mpjpe'])
    all_pjpe.append(lit_model.test_history[0]['pjpe'])
    all_activity_mpjpe.append(lit_model.test_history[0]['activities_mpjpe'])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model_single_actor/actor_vp1/lightning_logs


train_dataset 2651 val_dataset 5103 test_dataset 8414


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type          | Params
----------------------------------------
0 | model | BaselineModel | 4.3 M 
----------------------------------------
4.3 M     Trainable params
0         Non-trainable params
4.3 M     Total params
17.105    Total estimated model params size (MB)
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (41) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(
Restoring states from the checkpoint path at saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model_single_actor/actor_vp1/lightning_logs/version_0/checkpoints/epoch=79-step=3280.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model_single_actor/actor_vp1

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           mpjpe           │     85.07976112604777     │
└───────────────────────────┴───────────────────────────┘

skipping problematic image 15783
skipping problematic image 17258
skipping problematic image 17259


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           mpjpe           │    233.28722259526242     │
└───────────────────────────┴───────────────────────────┘

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model_single_actor/actor_vp10/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type          | Params
----------------------------------------
0 | model | BaselineModel | 4.3 M 
----------------------------------------
4.3 M     Trainable params
0         Non-trainable params
4.3 M     Total params
17.105    Total estimated model params size (MB)


train_dataset 2748 val_dataset 5103 test_dataset 8414


/opt/conda/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (42) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(
Restoring states from the checkpoint path at saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model_single_actor/actor_vp10/lightning_logs/version_0/checkpoints/epoch=89-step=3780.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model_single_actor/actor_vp10/lightning_logs/version_0/checkpoints/epoch=89-step=3780.ckpt


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           mpjpe           │    128.43140861488035     │
└───────────────────────────┴───────────────────────────┘

In [6]:
pd.DataFrame(all_mpjpe, index=all_actors)

,0
vp1,85.079761
vp2,92.784538
vp3,89.206783
vp4,92.451190
vp5,80.480865
vp6,146.580001
vp7,92.206528
vp8,126.586498
vp9,233.287223
vp10,128.431409


In [27]:
pd.DataFrame(all_mpjpe, index=all_actors).mean()

0    116.70948
dtype: float64

In [17]:
pjpe_df = pd.concat(all_pjpe, axis=1)
pjpe_df.columns = all_actors
pjpe_df

,vp1,vp2,vp3,vp4,vp5,vp6,vp7,vp8,vp9,vp10
nose,66.997449,80.114142,76.501030,77.363411,65.273843,86.975422,72.267712,100.775779,222.808078,100.035530
left_eye,56.997009,67.130475,67.536875,66.178774,56.176690,54.024487,60.893589,101.967055,169.223583,86.035619
right_eye,56.469500,68.196346,65.860015,71.123830,56.374316,83.425059,61.563346,79.559751,178.759843,97.154303
left_ear,29.983005,18.097908,28.196803,34.017291,13.406121,32.368408,24.026369,43.739742,74.109291,32.385681
right_ear,58.776419,61.951644,74.635509,72.333260,49.683760,97.084713,64.093910,88.971363,198.107857,112.134091
left_shoulder,82.712661,92.758853,72.191431,76.158628,80.523983,157.872522,78.888357,120.497075,221.370930,117.818317
right_shoulder,60.154104,82.300347,55.883959,72.450738,54.183461,154.832678,65.077839,92.829945,196.693866,102.536244
left_elbow,93.456873,107.064101,106.624408,99.923935,100.496998,204.258497,102.914147,129.512160,238.515928,127.836907
right_elbow,75.862178,94.486304,77.767349,88.179875,81.149053,125.824343,95.382537,115.473064,189.322487,125.185607
left_wrist,97.270754,102.259558,123.491146,115.815558,97.228228,134.803598,117.611211,144.922277,218.018718,127.146398


In [22]:
pd.DataFrame(pjpe_df.mean(axis=1), columns=['average_pjpe'])

,average_pjpe
nose,94.911240
left_eye,78.616416
right_eye,81.848631
left_ear,33.033062
right_ear,87.777253
left_shoulder,110.079276
right_shoulder,93.694318
left_elbow,131.060395
right_elbow,106.863280
left_wrist,127.856745


In [31]:
pd.DataFrame(all_activity_mpjpe, index=all_actors).T.sort_index()

,vp1,vp2,vp3,vp4,vp5,vp6,vp7,vp8,vp9,vp10
closing_bottle,83.962172,94.440625,85.978059,80.108025,77.673368,162.603370,85.418168,136.402805,239.377809,106.266509
closing_door_inside,123.239086,132.366302,110.901464,124.461042,107.147467,145.200579,NaN,107.274490,319.383381,167.016442
closing_laptop,70.552554,NaN,NaN,87.207866,80.155923,148.843459,85.697271,117.574734,166.472963,88.250450
drinking,89.912385,86.890143,82.076470,82.938764,78.139435,154.881497,81.509431,124.415249,184.530203,107.284948
eating,72.078443,90.061661,82.488850,91.689739,62.495712,87.785313,79.332264,91.558193,211.338378,143.494335
entering_car,164.155579,167.124382,141.170789,171.802700,143.017638,198.302013,171.087645,240.179887,414.280399,202.795484
exiting_car,260.060265,251.529397,246.948276,217.035197,180.625401,373.971637,294.885384,609.532586,NaN,251.295928
fastening_seat_belt,104.136150,106.794444,98.046647,106.494314,93.184500,148.713038,97.562420,124.307150,225.264651,135.522457
fetching_an_object,141.518177,133.695717,146.984259,139.839784,120.536205,167.473286,142.714006,213.523396,330.641175,179.484914
interacting_with_phone,64.396745,80.941372,78.151639,89.414623,77.519166,107.265009,82.697650,103.796471,190.394481,133.278870


In [33]:
pd.DataFrame(pd.DataFrame(all_activity_mpjpe, index=all_actors).T.mean(axis=1), columns=['pjpe']).sort_index()

,pjpe
closing_bottle,115.223091
closing_door_inside,148.554473
closing_laptop,105.594402
drinking,107.257852
eating,101.232289
entering_car,201.391652
exiting_car,298.431563
fastening_seat_belt,124.002577
fetching_an_object,171.641092
interacting_with_phone,100.785603
